In [1]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import  DataLoader,random_split
from torchvision import transforms, datasets
from torch.utils.tensorboard import SummaryWriter

t=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,),(0.5,))])
train=datasets.MNIST(root='./data',train=True,download=True,transform=t)
test=datasets.MNIST(root='./data',train=False,download=True,transform=t)
train.targets[5]

total_len=len(train)
training_size=int(0.8*total_len)
valid_size=int(0.2*total_len)
print(training_size,valid_size)
trainSet,valSet=random_split(train,[training_size,valid_size])

train_loader = DataLoader(trainSet, batch_size=64, shuffle=True)
val_loader = DataLoader(valSet, batch_size=64, shuffle=False)
test_loader = DataLoader(test, batch_size=64, shuffle=False)
dataiter = iter(train_loader)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
writer=SummaryWriter()
import torch
trainloss=[]
valoss=[]
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        #print(labels.size())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            vloss = criterion(outputs, labels)
            val_loss +=criterion(outputs, labels).item()

        val_loss /= len(val_loader)
        writer.add_scalar('loss',val_loss,global_step=epoch)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {val_loss:.4f}")
    trainloss.append(loss.data)
    valoss.append(vloss.data)




48000 12000
Epoch 1/5, Validation Loss: 0.2618
Epoch 2/5, Validation Loss: 0.1848
Epoch 3/5, Validation Loss: 0.1480
Epoch 4/5, Validation Loss: 0.1475
Epoch 5/5, Validation Loss: 0.1350


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader,random_split
from torchvision import transforms
import tensorflow as tf
class CustomDataset(Dataset):
    def __init__(self,root_dir,transform=None):
        self.root_dir=root_dir
        self.transform=transform
        self.file_paths=[]
        self.class_names=sorted(os.listdir(root_dir))

        for class_name in self.class_names:
            class_path=os.path.join(self.root_dir,class_name)
            files=[os.path.join(class_path, file) for file in os.listdir(class_path)]
            self.file_paths.extend(files)

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self,idx):
        img_path=self.file_paths[idx]
        image=Image.open(img_path)

        if self.transform:
            image=self.transform(image)

        class_name=os.path.basename(os.path.dirname(img_path))
        label_idx=torch.tensor(self.class_names.index(class_name),dtype=torch.long)

        return image,label_idx,class_name
    
transform=transforms.Compose([transforms.Resize((256,256)),transforms.ToTensor()])
dataset=CustomDataset(root_dir=r"",transform=transform)
print("Length of dataset:",len(dataset))
index=1
img,label_idx,label_name=dataset[index]
import numpy as np
img = np.reshape(img, (-1,256,256,3))
file_writer = tf.summary.create_file_writer("./logs")
with file_writer.as_default():
  tf.summary.image("Training data", img, step=0)
train_size=int(0.8*len(dataset))
test_size=len(dataset)-train_size
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])
class RBFLayer(torch.nn.Module):
    def __init__(self,n,input_dim):
        super(RBFLayer,self).__init__()
        self.centers=torch.nn.Parameter(torch.randn(n,input_dim))
        self.sigma=torch.nn.Parameter(torch.ones(n))
    def forward(self,x):
        x=x.view(x.size(0),-1)
        dist=torch.cdist(x,self.centers)
        rbf=torch.exp(-dist.pow(2)/(2*self.sigma.pow(2)))
        return rbf
class RBFNetwork(torch.nn.Module):
    def __init__(self,input_dim,n,od):
        super(RBFNetwork,self).__init__()
        self.rbflayer=RBFLayer(n,input_dim)
        self.linearlayer=torch.nn.Linear(n,od)
    def forward(self,x):
        x=self.rbflayer(x)
        op=self.linearlayer(x)
        return op
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
input_dim=3*256*256
od=2
n=50
model=RBFNetwork(input_dim,n,od)
cri=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
writer=SummaryWriter()
dataiter = iter(train_loader)
trainloss=[]
val_loss=0.0
epoch=5
for i in range(epoch):
    for images,label_idx,labels in train_loader:
        optimizer.zero_grad()
        output=model(images)
        #print(label_idx)
        loss=cri(output,label_idx)
        loss.backward()
        optimizer.step()
        val_loss +=cri(output, label_idx).item()

    val_loss /= len(train_loader)
    writer.add_scalar('loss',val_loss,global_step=epoch)
    print("epoch ",i+1,loss.item())
    trainloss.append(loss.data)


In [ ]:
import tensorflow as tf                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
bs=32
ims=(32,32)
data=ImageDataGenerator(rescale=1./255)
train_gen=data.flow_from_directory(r"c:\Users\maha9\OneDrive\Documents\datasets\plants\Apple\Train",
                                   target_size=ims,
                                   batch_size=bs,
                                   class_mode="categorical",
                                   shuffle=True)

Found 7771 images belonging to 4 classes.


In [ ]:
import tensorflow
from sklearn.cluster import KMeans
from tensorflow.keras import layers,models
test_gen=data.flow_from_directory(r"c:\Users\maha9\OneDrive\Documents\datasets\plants\Apple\Test",
                                   target_size=ims,
                                   batch_size=bs,
                                   class_mode="categorical",
                                   shuffle=False)
num_centers=4
kmeansdata=[]
for i in range(len(train_gen)):
    inputs,_=train_gen.next()
    kmeansdata.append(inputs.reshape(inputs.shape[0],-1))
    import numpy as np
kmeansdata=np.concatenate(kmeansdata,axis=0)
kmeans=KMeans(n_clusters=num_centers,random_state=42)
kmeans.fit(kmeansdata)
initial_centers=kmeans.cluster_centers_
class RBFLayer(layers.Layer):
    def __init__(self, output_dim):
        self.output_dim = output_dim
        super(RBFLayer, self).__init__()

    def build(self, input_shape):
        centers = initial_centers

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[-1]),
                                       initializer=tensorflow.constant_initializer(centers),
                                       trainable=True)
        super(RBFLayer, self).build(input_shape)
    def call(self, x):
        diff = tensorflow.expand_dims(x, axis=1) - self.centers
        norm = tensorflow.norm(diff, axis=-1)
        sigma=tensorflow.math.reduce_std(x)
        return tensorflow.exp(-0.1 * tensorflow.square(norm)/(2*(sigma**2)))
model=models.Sequential([
    layers.Flatten(input_shape=(32, 32, 3)), 
    RBFLayer(4), 
    layers.Dense(4,activation='softmax')  
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tensor_callback=TensorBoard(log_dir="./logs")
model.fit(train_gen,epochs=5,validation_data=test_gen,callbacks=[tensor_callback])

Found 196 images belonging to 4 classes.


c:\Users\maha9\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(




Epoch 1/5


243/243 [==============================] - 7s 25ms/step - loss: 1.3858 - accuracy: 0.2516 - val_loss: 1.3850 - val_accuracy: 0.2602
Epoch 2/5
243/243 [==============================] - 5s 22ms/step - loss: 1.3851 - accuracy: 0.2578 - val_loss: 1.3846 - val_accuracy: 0.2602
Epoch 3/5
243/243 [==============================] - 5s 23ms/step - loss: 1.3849 - accuracy: 0.2563 - val_loss: 1.3846 - val_accuracy: 0.2602
Epoch 4/5
243/243 [==============================] - 6s 23ms/step - loss: 1.3850 - accuracy: 0.2512 - val_loss: 1.3845 - val_accuracy: 0.2602
Epoch 5/5
243/243 [==============================] - 6s 23ms/step - loss: 1.3849 - accuracy: 0.2518 - val_loss: 1.3845 - val_accuracy: 0.2602


tensorboard --logdir=logs